**Entrenamiento Modelo para detectar matrículas**

In [ ]:
from ultralytics import YOLO

# --- 1. Cargar el modelo base yolo11n.pt ---
try:
    model = YOLO('yolo11n.pt') # Carga los pesos base de yolo11n
    print("Modelo base 'yolo11n.pt' cargado correctamente.")
except Exception as e:
    print(f"Error cargando el modelo base 'yolo11n.pt': {e}")
    print("Asegúrate de que el archivo existe y es compatible con Ultralytics.")
    exit()

# --- 2. Iniciar Entrenamiento ---
results = model.train(
    data='matriculas.yaml',    # Ruta al archivo YAML (solo 1 clase: matricula)
    epochs=70,                    # Número de épocas
    imgsz=640,                    # Tamaño de imagen
    batch=4,                      # Tamaño del lote
    name='modelo_matriculas_yolo11', # Nombre para la carpeta de resultados
    exist_ok=True                 # Permite sobrescribir
)

print("\n¡Entrenamiento para SOLO matrículas (base yolo11n) completado!")
# La ruta real donde se guardan los pesos
print(f"Mejores pesos del modelo guardados en: runs/detect/{results.save_dir}/weights/best.pt")

Modelo base 'yolo11n.pt' cargado correctamente.
New https://pypi.org/project/ultralytics/8.3.223 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.215  Python-3.9.23 torch-2.8.0+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=matriculas.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=70, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=mo

**Validación Visual: Detección Combinada de vehículos y matrículas**

In [ ]:
from ultralytics import YOLO
import cv2
import glob # Para encontrar archivos que coincidan con un patrón (ej: *.jpg)
import os # Para crear carpetas y manejar rutas de archivos

# --- 1. CONFIGURACIÓN DE MODELOS ---

# Cargamos el modelo general YOLOv8 pre-entrenado (para vehículos)
try:
    # 'yolov8n.pt' es un modelo "nano" oficial, entrenado en el dataset COCO.
    # Lo usaremos para detectar objetos comunes como coches.
    model_general_coches = YOLO('yolov8n.pt')
    print("Modelo general (yolov8n.pt) para coches cargado correctamente.")
except Exception as e:
    print(f"Error cargando modelo general 'yolov8n.pt': {e}")
    exit()

# Cargamos el modelo yolo11 entrenado
try:
    ruta_modelo_matriculas_entrenado = 'runs/detect/modelo_matriculas_yolo11/weights/best.pt'
    model_matriculas = YOLO(ruta_modelo_matriculas_entrenado)
    print(f"Modelo entrenado para matrículas '{ruta_modelo_matriculas_entrenado}' cargado correctamente.")
except Exception as e:
    print(f"Error cargando tu modelo entrenado de matrículas '{ruta_modelo_matriculas_entrenado}': {e}")
    print("Asegúrate de que la ruta apunta al archivo 'best.pt' correcto.")
    exit()

# --- 2. CONFIGURACIÓN DE PARÁMETROS ---

# Clases de interés del modelo general (vehículos COCO)
# Le decimos al modelo general que SÓLO nos interesan estas clases:
# 2='car', 3='motorcycle', 5='bus', 7='truck'
coco_vehicle_classes = [2, 3, 5, 7]

# Colores y confianzas
color_vehiculo = (255, 0, 0) # Azul
color_matricula = (0, 255, 0) # Verde
conf_vehiculo = 0.40 # Umbral de confianza para vehículos (ignora si es menor a 40%)
conf_matricula = 0.1 # Umbral de confianza para matrículas (ignora si es menor a 10%)

# --- 3. DEFINIR IMÁGENES DE PRUEBA Y SALIDA ---

# glob.glob() crea una lista de todos los archivos que coinciden con el patrón
ruta_carpeta_imagenes_test = './TGC_RBNW/imgenes_comprobar/*'
rutas_imagenes_test = glob.glob(ruta_carpeta_imagenes_test)
# Comprobación de seguridad: si no encuentra imágenes, se detiene.
if not rutas_imagenes_test:
    print(f"Error: No se encontraron imágenes en '{ruta_carpeta_imagenes_test}'.")
    exit()
# Define el nombre de la carpeta donde se guardarán los resultados
output_dir_deteccion_combinada = 'resultados2_deteccion_combinada_y11base'
# os.makedirs() crea la carpeta. exist_ok=True evita que dé error si la carpeta ya existe.
os.makedirs(output_dir_deteccion_combinada, exist_ok=True)

print(f"Procesando {len(rutas_imagenes_test)} imágenes...")
print(f"Resultados guardados en: {output_dir_deteccion_combinada}")

# --- 4. PROCESAR IMÁGENES Y DIBUJAR ---

# Recorremos cada ruta de imagen encontrada
for img_path in rutas_imagenes_test:
    base_filename = os.path.basename(img_path)
    # print(f"Procesando: {base_filename}")

    # Cargamos la imagen desde el disco usando OpenCV
    frame = cv2.imread(img_path)
    if frame is None: continue
    # Crea una COPIA de la imagen original para dibujar sobre ella.
    frame_annotated = frame.copy()

    # --- 4.1 Detectar VEHÍCULOS ---
    # Ejecutamos el modelo general sobre la imagen
    results_coches = model_general_coches.predict(frame, classes=coco_vehicle_classes, conf=conf_vehiculo, verbose=False)
    if len(results_coches) > 0 and len(results_coches[0].boxes) > 0:
        # Iteramos sobre cada vehículo detectado
        for box_veh in results_coches[0].boxes:
            # Extraemos coordenadas, confianza y clase
            coords_veh = box_veh.xyxy.cpu().numpy().astype(int)[0]
            x1_v, y1_v, x2_v, y2_v = coords_veh
            conf_v = box_veh.conf.cpu().numpy()[0]
            class_id_v = int(box_veh.cls.cpu().numpy()[0])
            label_veh = model_general_coches.names[class_id_v]
            # Dibujamos el rectángulo azul
            cv2.rectangle(frame_annotated, (x1_v, y1_v), (x2_v, y2_v), color_vehiculo, 2)
            # Escribimos el texto (etiqueta + confianza)
            cv2.putText(frame_annotated, f'{label_veh} {conf_v:.2f}', (x1_v, y1_v - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_vehiculo, 2)

    # --- 4.2 Detectar MATRÍCULAS ---
    # Ejecutamos el modelo entrenado para las matrículas sobre la imagen
    results_matricula = model_matriculas.predict(frame, conf=conf_matricula, verbose=False)
    # Comprobamos si el modelo encontró algo
    if len(results_matricula) > 0 and len(results_matricula[0].boxes) > 0:
        # Iteramos sobre cada matrícula detectada
        for box_mat in results_matricula[0].boxes:
            # Extraemos coordenadas y confianza
            coords_mat = box_mat.xyxy.cpu().numpy().astype(int)[0]
            mx1, my1, mx2, my2 = coords_mat
            conf_m = box_mat.conf.cpu().numpy()[0]
            label_mat = model_matriculas.names[0] # Siempre será 'matricula'
            # Dibujamos el rectángulo verde
            cv2.rectangle(frame_annotated, (mx1, my1), (mx2, my2), color_matricula, 2)
            # Escribimos el texto (etiqueta + confianza)
            cv2.putText(frame_annotated, f'{label_mat} {conf_m:.2f}', (mx1, my1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_matricula, 2)

    # --- GUARDAR ---
    output_path = os.path.join(output_dir_deteccion_combinada, f"comb_{base_filename}")
    # Guardamos la imagen (la copia) con los rectángulos dibujados en el disco
    cv2.imwrite(output_path, frame_annotated)

print("\nProceso de detección combinada completado.")
print(f"Resultados guardados en: {output_dir_deteccion_combinada}")

Modelo general (yolov8n.pt) para coches cargado correctamente.
Modelo entrenado para matrículas 'runs/detect/modelo_matriculas_yolo11/weights/best.pt' cargado correctamente.
Procesando 6 imágenes...
Resultados guardados en: resultados2_deteccion_combinada_y11base

Proceso de detección combinada completado.
Resultados guardados en: resultados2_deteccion_combinada_y11base


**EASYOCR Y TESSERACT**

In [ ]:
import cv2
import os
import csv
import glob
import time
from ultralytics import YOLO
import easyocr
import pytesseract

# --- 1. CONFIGURACIÓN INICIAL Y CARGA DE MODELOS ---

# --- Cargar Modelos YOLO ---
print("Cargando modelos YOLO...")
try:
    # Modelo general para detectar vehículos
    model_general_coches = YOLO('yolov8n.pt')
    # Modelo personalizado para detectar matrículas
    model_matriculas = YOLO('runs/detect/modelo_matriculas_yolo11/weights/best.pt')
    print("Modelos YOLO cargados.")
except Exception as e:
    print(f"Error cargando modelos YOLO: {e}")
    exit()

# --- Cargar EasyOCR ---
print("Cargando EasyOCR...")
try:
    # Inicializa el lector de EasyOCR para español e inglés, usando CPU
    reader_easyocr = easyocr.Reader(['es', 'en'], gpu=False)
    print("EasyOCR cargado correctamente (CPU).")
except Exception as e:
    print(f"Error cargando easyOCR: {e}")
    exit()

# --- Cargar Tesseract ---
print("Cargando Tesseract...")
try:
    # Indicamos a Python dónde encontrar el archivo 'tesseract.exe'
    pytesseract.pytesseract.tesseract_cmd = r'C:\Users\lucia\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'
    version = pytesseract.get_tesseract_version()
    print(f"Tesseract {version} cargado correctamente.")
except Exception as e:
    print(f"Error cargando Tesseract. ¿Instalaste el programa y configuraste bien la ruta?")
    print(f"Error: {e}")
    exit() 

# --- Configuración de Tesseract para matrículas ---
# --psm 7: Trata la imagen como una sola línea de texto y solo permite los caracteres especificados.
config_tesseract = '--psm 7 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNPQRSTUVWXYZ'


# --- 2. CONFIGURACIÓN DE RUTAS Y CSV ---

# --- Rutas de entrada y salida ---
# Buscamos las imágenes en esta carpeta
ruta_carpeta_imagenes_test = './TGC_RBNW/imgenes_comprobar/*'
rutas_imagenes_test = glob.glob(ruta_carpeta_imagenes_test)
# Carpeta donde se guardarán las imágenes anotadas y el CSV
output_dir_deteccion_combinada = 'resultados_comparativa_OCR'
os.makedirs(output_dir_deteccion_combinada, exist_ok=True)

# --- Configuración CSV ---
# Definimos la ruta completa del archivo CSV de resultados
csv_output_path = os.path.join(output_dir_deteccion_combinada, 'comparativa_ocr_results.csv')
# Lista para almacenar temporalmente los resultados de cada imagen
csv_results = []
# Columnas para el archivo CSV.
csv_header = [
    'Imagen', 
    'Conf_Deteccion', 
    'Matricula_EasyOCR', 
    'Tiempo_EasyOCR_ms', 
    'Matricula_Tesseract', 
    'Tiempo_Tesseract_ms'
]

print(f"Procesando {len(rutas_imagenes_test)} imágenes para EasyOCR y Tesseract...")


# --- 3. PROCESAR IMÁGENES ---

# Procesamos imagen por imagen
for img_path in rutas_imagenes_test:
    base_filename = os.path.basename(img_path) # Obtiene el nombre
    frame = cv2.imread(img_path) # Carga la imagen
    if frame is None: continue # Si la imagen está corrupta, la salta
    frame_annotated = frame.copy() # Copia la imagen para dibujar sobre ella

    # --- 3.1. Detectar VEHÍCULOS ---
    results_coches = model_general_coches.predict(frame, classes=[2, 3, 5, 7], conf=0.40, verbose=False)
    if len(results_coches) > 0 and len(results_coches[0].boxes) > 0:
        for box_veh in results_coches[0].boxes:
            coords_veh = box_veh.xyxy.cpu().numpy().astype(int)[0]
            x1_v, y1_v, x2_v, y2_v = coords_veh
            # Dibuja un rectángulo azul para el coche
            cv2.rectangle(frame_annotated, (x1_v, y1_v), (x2_v, y2_v), (255, 0, 0), 2) # Azul para coche

    # --- 3.2. Detectar MATRÍCULAS ---
    # Ejecutamos el modelo personalizado de matrículas
    results_matricula = model_matriculas.predict(frame, conf=0.1, verbose=False)
    
    if len(results_matricula) > 0 and len(results_matricula[0].boxes) > 0:
        # Iteramos sobre cada matrícula detectada en la imagen
        for box_mat in results_matricula[0].boxes:
            coords_mat = box_mat.xyxy.cpu().numpy().astype(int)[0]
            mx1, my1, mx2, my2 = coords_mat
            conf_m = box_mat.conf.cpu().numpy()[0]
            
            # # Recorta la región de la matrícula (con un pequeño margen de 5px)
            matricula_recortada = frame[
                max(0, my1 - 5) : min(frame.shape[0], my2 + 5), 
                max(0, mx1 - 5) : min(frame.shape[1], mx2 + 5)
            ]
            
            # Si el recorte está vacío (error de coordenadas), saltar
            if matricula_recortada.size == 0:
                print(f"  > {base_filename} - Recorte de matrícula vacío, saltando.")
                continue

            # --- 3.3. Ejecutar EasyOCR ---
            texto_easyocr = "N/A"
            tiempo_easyocr = 0.0
            try:
                t_start_e = time.time() # Inicia cronómetro
                # Ejecuta EasyOCR sobre el recorte
                ocr_result = reader_easyocr.readtext(matricula_recortada, detail=0, allowlist='0123456789ABCDEFGHIJKLMNPQRSTUVWXYZ')
                t_end_e = time.time() # Detiene cronómetro
                tiempo_easyocr = (t_end_e - t_start_e) * 1000 # Calcula tiempo en milisegundos
                if ocr_result:
                    # Limpia el texto: mayúsculas y sin espacios
                    texto_easyocr = ocr_result[0].upper().replace(" ", "")
            except Exception as e:
                print(f"Error en EasyOCR: {e}")

            # --- 3.4. Ejecutar Tesseract ---
            texto_tesseract = "N/A"
            tiempo_tesseract = 0.0
            try:
                t_start_t = time.time() # Iniciamos cronómetro
                # Ejecutamos Tesseract sobre el MISMO recorte
                texto_tesseract = pytesseract.image_to_string(matricula_recortada, config=config_tesseract)
                t_end_t = time.time() # Detiene cronómetro
                tiempo_tesseract = (t_end_t - t_start_t) * 1000 # Calculamos tiempo en ms
                # Limpiamos resultado de Tesseract
                texto_tesseract = texto_tesseract.strip().upper().replace(" ", "")
                if not texto_tesseract:
                    texto_tesseract = "N/A"
            except Exception as e:
                print(f"Error en Tesseract: {e}")
                
            # --- 3.5. Guardar resultado UNIFICADO en la lista ---
            # Añadimos una nueva fila a nuestros resultados
            csv_results.append([
                base_filename, 
                f"{conf_m:.2f}", 
                texto_easyocr, 
                f"{tiempo_easyocr:.1f}", 
                texto_tesseract, 
                f"{tiempo_tesseract:.1f}"
            ])
            
            # --- 3.6. Dibujar AMBOS resultados en la imagen ---
            # Caja verde para la matrícula
            cv2.rectangle(frame_annotated, (mx1, my1), (mx2, my2), (0, 255, 0), 2)
            
            # Texto de EasyOCR (arriba, verde)
            label_easyocr = f'EasyOCR: {texto_easyocr} ({tiempo_easyocr:.1f}ms)'
            cv2.putText(frame_annotated, label_easyocr, (mx1, my1 - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            
            # Texto de Tesseract (debajo, magenta)
            label_tesseract = f'Tesseract: {texto_tesseract} ({tiempo_tesseract:.1f}ms)'
            cv2.putText(frame_annotated, label_tesseract, (mx1, my1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
            
            print(f"  > {base_filename} [EasyOCR: {texto_easyocr} | Tesseract: {texto_tesseract}]")

    # --- 3.7. Guardar la imagen anotada ---
    output_path = os.path.join(output_dir_deteccion_combinada, f"comparativa_{base_filename}")
    cv2.imwrite(output_path, frame_annotated) # Guardamos la imagen con los dibujos

# --- 4. GUARDAR EL ARCHIVO CSV UNIFICADO ---
# Una vez finalizado el bucle, escribimos TODOS los resultados en el CSV
try:
    with open(csv_output_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(csv_header) # Escribir el header unificado
        writer.writerows(csv_results) # Escribir todos los resultados
except Exception as e:
    print(f"Error al guardar el CSV: {e}")

print(f"\n--- Proceso UNIFICADO completado ---")
print(f"Resultados CSV guardados en: {csv_output_path}")
print(f"Imágenes anotadas guardadas en: {output_dir_deteccion_combinada}")

Using CPU. Note: This module is much faster with a GPU.


Cargando modelos YOLO...
Modelos YOLO cargados.
Cargando EasyOCR...
EasyOCR cargado correctamente (CPU).
Cargando Tesseract...
Tesseract 5.4.0.20240606 cargado correctamente.
Procesando 10 imágenes para EasyOCR y Tesseract...
  > 0116GPD.jpeg [EasyOCR: 0116GPD | Tesseract: F0TT6GPD]
  > 2753LBJ.jpeg [EasyOCR: 2753LBJ | Tesseract: N/A]
  > 3129JRM.jpeg [EasyOCR: 3129JRM | Tesseract: N/A]
  > 3129JRM.jpeg [EasyOCR: TAXI | Tesseract: TAXI]
  > 3168JHM.jpeg [EasyOCR: E | Tesseract: 13168JHM]
  > 3654MYH.jpeg [EasyOCR: 3654MYH | Tesseract: N/A]
  > 4334NFH.jpeg [EasyOCR: 14334NFH | Tesseract: 94334NFH]
  > 8389CK.jpeg [EasyOCR: GC8389CK | Tesseract: GC8389CK]
  > 8389CK.jpeg [EasyOCR: N/A | Tesseract: N/A]
  > 8996JNV.jpg [EasyOCR: JNV | Tesseract: N/A]
  > 8996JNV.jpg [EasyOCR: 3232JCJ | Tesseract: 3232NC9]
  > 8996JNV.jpg [EasyOCR: N/A | Tesseract: KA]
  > GX497.jpeg [EasyOCR: GGAGX7497 | Tesseract: N/A]
  > GX497.jpeg [EasyOCR: N/A | Tesseract: N/A]
  > PGC7085.jpg [EasyOCR: JPGC7085N | 

**Detección de matrículas y vehículos en vídeo**

In [ ]:
import cv2
import os
import csv
from ultralytics import YOLO
import easyocr

# --- 1. CONFIGURACIÓN INICIAL ---

# Cargar Modelos YOLO
print("Cargando modelos YOLO...")
try:
    # Modelo general para detectar personas y vehículos
    model_general_coches = YOLO('yolov8n.pt')
    # Modelo personalizado entrenado solo para matrículas
    ruta_modelo_matriculas = 'runs/detect/modelo_matriculas_yolo11/weights/best.pt'
    model_matriculas = YOLO(ruta_modelo_matriculas)
    print("Modelos YOLO cargados.")
except Exception as e:
    print(f"Error cargando modelos YOLO: {e}")
    exit()

# Cargar EasyOCR
print("Cargando EasyOCR (esto puede tardar un momento)...")
try:
    reader_easyocr = easyocr.Reader(['es', 'en'], gpu=False)
    print("EasyOCR cargado.")
except Exception as e:
    print(f"Error cargando easyOCR: {e}")
    exit()

# Clases de interés: 0=person, 2=car, 3=motorcycle, 5=bus, 7=truck
coco_classes_of_interest = [0, 2, 3, 5, 7] 
# Clases que son vehículos (para buscar matrículas dentro)
vehicle_classes = [2, 3, 5, 7] 

# Colores y confianzas
color_vehiculo_persona = (255, 0, 0) # Azul
color_matricula = (0, 255, 0) # Verde
conf_vehiculo = 0.40
conf_matricula = 0.1

# --- 2. CONFIGURACIÓN DE ENTRADA Y SALIDA ---

# Rutas de Vídeo
ruta_video_entrada = './input_media/video.mp4'
ruta_video_salida = './resultados_video/video_final_con_ocr_asociado.mp4'
os.makedirs('./resultados_video', exist_ok=True)

# Rutas de CSV
csv_output_path = './resultados_video/log_detecciones_asociado.csv'
# Definimos header
csv_header = [
    'fotograma', 
    'tipo_objeto',      # 'car', 'person', etc.
    'confianza',        # Confianza del objeto (0-1)
    'identificador_tracking', # ID del objeto
    'x1', 'y1', 'x2', 'y2', # Coordenadas del objeto
    'matricula_en_su_caso', # 'matricula' o 'N/A'
    'confianza_matricula',  # Confianza de la matrícula (0-1)
    'mx1', 'my1', 'mx2', 'my2', # Coordenadas de la matrícula
    'texto_matricula'   # Texto de la matrícula
]

# --- 3. INICIALIZAR LECTORES Y ESCRITORES ---

# Lector de Video
cap = cv2.VideoCapture(ruta_video_entrada)
if not cap.isOpened():
    print(f"Error: No se pudo abrir el video '{ruta_video_entrada}'")
    exit()

# Escritor de Video
# Obtenemos las propiedades del vídeo original para que el vídeo de salida coincida(ancho, alto y fps).
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Define el códec de vídeo. 'mp4v'
# Crea el objeto "escritor" de vídeo con la ruta de salida, códec, FPS y tamaño.
out = cv2.VideoWriter(ruta_video_salida, fourcc, fps, (frame_width, frame_height))

# Escritor de CSV
try:
    # Abre el archivo CSV en modo escritura
    csv_file = open(csv_output_path, 'w', newline='', encoding='utf-8')
    # Crea un objeto "escritor" de CSV
    csv_writer = csv.writer(csv_file)
    # Escribe la fila del encabezado (definida arriba) en el archivo.
    csv_writer.writerow(csv_header)
except Exception as e:
    print(f"Error al abrir el CSV: {e}")
    exit()

# Variables de conteo y tracking (Método de IDs únicos)
frame_num = 0 # Contador de frames
# Usamos 'sets' para contar solo los IDs únicos que aparecen
tracked_ids = {
    'person': set(),
    'car': set(),
    'motorcycle': set(),
    'bus': set(),
    'truck': set()
}

# --- Función auxiliar para calcular Intersección sobre Unión (IoU) ---
# Esta función nos dirá qué tan superpuesta está una caja (matrícula) con otra (coche)
# La usaremos para ver si una matrícula está "dentro" de un coche.
# La usaremos para ver si una matrícula está "dentro" de un coche.# Devuelve un valor entre 0 (sin superposición) y 1 (superposición perfecta).
def get_iou(boxA, boxB):
    # Determinar las coordenadas (x, y) de la intersección
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # Calcular el área de intersección
    interArea = max(0, xB - xA) * max(0, yB - yA)
    if interArea == 0:
        return 0

    # Calcular el área de ambas cajas
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # Calcular IoU: (Área de Intersección) / (Área Total - Área de Intersección)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

print(f"Procesando video: {ruta_video_entrada}...")
print("--- ¡Pulsa 'q' en la ventana emergente para salir! ---")

# --- 4. PROCESAR VIDEO FRAME A FRAME ---

while cap.isOpened():
    # `cap.read()`: Lee el siguiente fotograma del vídeo.
    # `ret`: Es un booleano (True/False) que indica si la lectura fue exitosa.
    # `frame`: Es la imagen (array de numpy) del fotograma actual.
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_num += 1 # Incrementamos el contador de fotograma.
    frame_annotated = frame.copy() # Creamos una COPIA del fotograma para dibujar sobre ella.

    # --- 4.1 Detectar y SEGUIR (Track) Vehículos y Personas ---
    # Usamos .track() para que el modelo asigne y mantenga IDs a los objetos.
    results_general = model_general_coches.track(
        frame,                          # La imagen original (sin dibujos).
        classes=coco_classes_of_interest, # Solo buscar las clases que nos interesan.
        conf=conf_vehiculo,               # Umbral de confianza.
        verbose=False,                  # No imprimir logs de YOLO en la consola.
        persist=True                    # Mantiene los IDs de tracking entre fotogramas.
    )

    # --- 4.2 Detectar Matrículas y hacer OCR ---
    # Primero detectamos TODAS las matrículas en el frame
    results_matricula = model_matriculas.predict(frame, conf=conf_matricula, verbose=False)
    
    # Almacenamos todas las matrículas detectadas en el fotograma y su OCR en una lista temporal
    detected_plates_data = []
    # Comprobamos si el modelo de matrículas encontró algo.
    if len(results_matricula) > 0 and len(results_matricula[0].boxes) > 0:
        # Iteramos sobre cada matrícula detectada en el fotograma.
        for box_mat in results_matricula[0].boxes:
            # Extraemos las coordenadas de la matrícula.
            coords_mat = box_mat.xyxy.cpu().numpy().astype(int)[0]
            mx1, my1, mx2, my2 = coords_mat
            # Extraemos la confianza de la detección.
            conf_m = box_mat.conf.cpu().numpy()[0]
            # Obtenemos el nombre de la clase (siempre será 'matricula').
            label_mat = model_matriculas.names[0] 

            # Recortar la matrícula para OCR
            # Añadimos un pequeño margen de 5 píxeles (si es posible) para ayudar al OCR.
            crop_matricula = frame[
                max(0, my1 - 5) : min(frame.shape[0], my2 + 5),
                max(0, mx1 - 5) : min(frame.shape[1], mx2 + 5)
            ]
            
            texto_ocr = "N/A"
            if crop_matricula.size > 0: 
                try:
                    # Ejecutamos EasyOCR sobre la imagen recortada.
                    # `detail=0`: Devuelve solo una lista de strings (no coordenadas).
                    # `allowlist`: Fuerza al OCR a solo reconocer estos caracteres.
                    ocr_result = reader_easyocr.readtext(crop_matricula, detail=0, allowlist='0123456789ABCDEFGHIJKLMNPQRSTUVWXYZ')
                    if ocr_result: # Si la lista no está vacía
                        # Tomamos el primer resultado, lo ponemos en mayúsculas y quitamos espacios.
                        texto_ocr = ocr_result[0].upper().replace(" ", "")
                except Exception as e:
                    # Si EasyOCR falla (ej. imagen muy pequeña), lo registramos
                    print(f"Error en EasyOCR en frame {frame_num}: {e}")
            
            # Guardamos los datos de esta matrícula en la lista termporal
            detected_plates_data.append({
                'coords': (mx1, my1, mx2, my2),
                'conf': conf_m,
                'label': label_mat,
                'text': texto_ocr
            })
            
            # --- Dibujamos la matrícula ---
            # Dibujamos la caja verde de la matrícula en la copia del fotograma.
            label_display_ocr = f'{texto_ocr} ({conf_m:.2f})'
            cv2.rectangle(frame_annotated, (mx1, my1), (mx2, my2), color_matricula, 2)
            cv2.putText(frame_annotated, label_display_ocr, (mx1, my1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_matricula, 2)


    # --- Tarea 3: Asociar Vehículos/Personas con Matrículas y Escribir CSV ---
    # Comprobamos si el tracker de objetos encontró algo.
    # Ahora, iteramos sobre los objetos SEGUIDOS (vehículos/personas)
    if results_general[0].boxes.id is not None:
        for box in results_general[0].boxes:
            # Coordenadas y datos del objeto (vehículo o persona)
            coords_v = box.xyxy.cpu().numpy().astype(int)[0]
            x1_v, y1_v, x2_v, y2_v = coords_v
            conf_v = box.conf.cpu().numpy()[0]
            class_id_v = int(box.cls.cpu().numpy()[0])
            track_id = int(box.id.cpu().numpy()[0]) # id de seguimiento
            label_veh = model_general_coches.names[class_id_v] # 'car', 'person', etc.

            # Conteo de IDs únicos
            if label_veh in tracked_ids:
                tracked_ids[label_veh].add(track_id)

            # --- Dibujar el objeto (Vehículo/Persona) ---
            # Dibujamos la caja azul del objeto en la copia del fotograma.
            label_display = f'ID: {track_id} ({label_veh})'
            cv2.rectangle(frame_annotated, (x1_v, y1_v), (x2_v, y2_v), color_vehiculo_persona, 2)
            cv2.putText(frame_annotated, label_display, (x1_v, y1_v - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_vehiculo_persona, 2)

            # --- Lógica de Asociación para el CSV ---
            # Creamos una lista con 7 valores por defecto (N/A) para las columnas de la matrícula.
            plate_info = ['N/A', 0.0, 0, 0, 0, 0, 'N/A']
            
            # Solo buscar matrículas si el objeto es un vehículo
            if class_id_v in vehicle_classes:
                best_iou = 0.0 # Para guardar el mejor IoU encontrado.
                best_plate = None # Para guardar la mejor matrícula encontrada.
                
                # Iteramos sobre la lista de matrículas que detectamos en la Tarea 2.
                for plate_data in detected_plates_data:
                    # Comprobar si la matrícula está DENTRO del vehículo
                    # Usamos IoU (Intersección sobre Unión)
                    iou = get_iou(coords_v, plate_data['coords'])
                    
                    # Si esta matrícula se superpone más que la "mejor" anterior...
                    if iou > best_iou:
                        best_iou = iou # ...actualizamos el mejor IoU.
                        best_plate = plate_data # ...guardamos esta matrícula.
                
                
                if best_plate is not None and best_iou > 0:
                    # ...reemplazamos la lista de 'N/A' con los datos reales de 'best_plate'.
                    plate_coords = best_plate['coords']
                    plate_info = [
                        best_plate['label'],
                        f"{best_plate['conf']:.2f}",
                        plate_coords[0], plate_coords[1], plate_coords[2], plate_coords[3],
                        best_plate['text']
                    ]

            # Escribir la fila completa en el CSV
            # Creamos una lista con los 8 primeros datos (del objeto)
            csv_writer.writerow([
                frame_num, label_veh, f"{conf_v:.2f}", track_id, 
                x1_v, y1_v, x2_v, y2_v,
                *plate_info # Desempaqueta la lista [label_mat, conf_m, mx1, ...]
            ])


    # --- 5. GUARDAR Y MOSTRAR ---
    
    # Guardar el fotograma procesado
    out.write(frame_annotated)
    
    # Mostrar el video en ventana emergente
    cv2.imshow('Deteccion (Vehiculos, Personas y OCR)', frame_annotated)

    # Esperar 1ms y comprobar si se pulsa la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Detenido por el usuario.")
        break

# --- 6. FINALIZACIÓN ---
print("\nLimpiando recursos...")
# Liberar los objetos de video
cap.release()
out.release()
# Cerrar el archivo CSV
csv_file.close()
# Cerrar todas las ventanas de OpenCV
cv2.destroyAllWindows()

# Imprimir el conteo final (IDs únicos)
print("\n--- Conteo Final (IDs únicos detectados) ---")
total_general = 0
for label, id_set in tracked_ids.items():
    count = len(id_set)
    print(f"Total {label}s: {count}")
    total_general += count
print(f"Total objetos seguidos: {total_general}")

print(f"\nProceso de video completado.")
print(f"Video guardado en: {ruta_video_salida}")
print(f"Log CSV guardado en: {csv_output_path}")

Using CPU. Note: This module is much faster with a GPU.


Cargando modelos YOLO...
Modelos YOLO cargados.
Cargando EasyOCR (esto puede tardar un momento)...
EasyOCR cargado.
Procesando video: ./input_media/video.mp4...
--- ¡Pulsa 'q' en la ventana emergente para salir! ---


c:\Users\lucia\anaconda33\envs\VC_P4\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Limpiando recursos...

--- Conteo Final (IDs únicos detectados) ---
Total persons: 32
Total cars: 164
Total motorcycles: 5
Total buss: 5
Total trucks: 18
Total objetos seguidos: 224

Proceso de video completado.
Video guardado en: ./resultados_video/video_final_con_ocr_asociado.mp4
Log CSV guardado en: ./resultados_video/log_detecciones_asociado.csv
